In [36]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord,match_coordinates_sky
from astroquery.gaia import Gaia
from astropy.coordinates import Angle
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"

In [37]:
## read CSC 2.1
path='/Users/baotong/Desktop/period_terzan5/CSC/'
filename='CSC2.1p_OIR_SDSSspecmatch.csv'
CSC=pd.read_csv(path+filename)
ra_CSC=CSC['ra'];dec_CSC=CSC['dec']
GAIA21P_source_id=CSC['GAIA21P_source_id']
GAIA21P_ra=CSC['GAIA21P_ra']
GAIA21P_dec=CSC['GAIA21P_dec']
Sep_GAIA21P_CSC21P=CSC['Sep_GAIA21P_CSC21P']
GAIA21P_g=CSC['GAIA21P_g']

/var/folders/z1/8lm05_ks6676c0ppy4pxnrlm0000gn/T/ipykernel_66974/4151308728.py:4: DtypeWarning: Columns (57,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  CSC=pd.read_csv(path+filename)


In [38]:
## read GC source info
result_all = pd.read_excel('/Users/baotong/Desktop/period_terzan5/candidate_allGC.xlsx', 'all')
ra_GC = np.array(result_all['ra'])
dec_GC = np.array(result_all['dec'])
seq_GC = np.array(result_all['seq'])
period = np.array(result_all['period_all'])
type = np.array(result_all['GC'])
dist = np.array(result_all['proj_dist'])
counts = np.array(result_all['counts'])
exptime = np.array(result_all['expT'])
L = np.array(result_all['L'])

In [39]:
## match GC source to CSC 2.1
c1 = SkyCoord(ra=ra_CSC * u.degree, dec=dec_CSC * u.degree)
c2 = SkyCoord(ra=ra_GC * u.degree, dec=dec_GC * u.degree)
idx, d2d, d3d = c2.match_to_catalog_sky(c1)
d2d=Angle(d2d)
d2d=d2d.arcsec

In [40]:
ra_GAIA=np.array(GAIA21P_ra[idx]);dec_GAIA=np.array(GAIA21P_dec[idx])
ID_GAIA=np.array(GAIA21P_source_id[idx])
dis_GC_CSC=d2d
ra_CSC=np.array(ra_CSC[idx]);dec_CSC=np.array(dec_CSC[idx])
header = ['seq', 'GCname', 'period','ra','dec','ra_CSC','dec_CSC','sep_GC_CSC','ra_GAIA','dec_GAIA',
          'sep_GC_GAIA','sep_CSC_GAIA','pmra','pmdec','pmra_GC','pmra_GC']

In [43]:
# 将数据转换为 DataFrame
data_list = []
for i in range(len(ra_GC)):
    # if np.isnan(ra_GAIA[i]):
    #     row=[seq_GC[i],type[i],period[i],ra_GC[i],dec_GC[i],ra_CSC[i],dec_CSC[i],dis_GC_CSC[i],
    #          0,0,0,0,0,0,0,0]
    #     data_list.append(row)
    #     continue
    # else:
        coord = SkyCoord(ra=ra_GC[i], dec=dec_GC[i], unit=(u.degree, u.degree), frame='icrs')
        width = u.Quantity(3, u.arcsec)
        height = u.Quantity(3, u.arcsec)
        r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
        if len(r)==0:
            row = [seq_GC[i], type[i], period[i], ra_GC[i], dec_GC[i], ra_CSC[i], dec_CSC[i], dis_GC_CSC[i],
                   0, 0, 0, 0, 0, 0, 0, 0]
            data_list.append(row)
            print('what',i)
            continue
        print(i,len(r),ra_CSC[i],dec_CSC[i])
        c1 = SkyCoord(ra=r[0]['ra'] * u.degree, dec=r[0]['dec'] * u.degree)
        c2 = SkyCoord(ra=ra_GC[i] * u.degree, dec=dec_GC[i] * u.degree)
        cc = SkyCoord(ra_CSC[i] * u.deg, dec_CSC[i] * u.deg, frame='fk5')
        dist1 = c1.separation(c2);dist1=dist1.arcsec
        dist2 = c1.separation(cc);dist2=dist2.arcsec
        row=[seq_GC[i],type[i],period[i],ra_GC[i],dec_GC[i],ra_CSC[i],dec_CSC[i],dis_GC_CSC[i],
             ra_GAIA[i],dec_GAIA[i],dist1,dist2,r[0]['pmra'],r[0]['pmdec'],0,0]
        data_list.append(row)
        # r.pprint(max_lines=12, max_width=130)
df = pd.DataFrame(data_list, columns=header)

INFO: Query finished. [astroquery.utils.tap.core]
what 0
INFO: Query finished. [astroquery.utils.tap.core]
1 1 276.1897613891948 -24.74277559826854
INFO: Query finished. [astroquery.utils.tap.core]
2 1 276.1877398364279 -24.890934332164623
INFO: Query finished. [astroquery.utils.tap.core]
what 3
INFO: Query finished. [astroquery.utils.tap.core]
4 1 276.1316716674979 -24.87151340444077
INFO: Query finished. [astroquery.utils.tap.core]
5 1 276.1354269833861 -24.868888804615537
INFO: Query finished. [astroquery.utils.tap.core]
6 2 276.1595156123924 -25.02702283135197
INFO: Query finished. [astroquery.utils.tap.core]
7 3 255.2984520756828 -30.11134066447533
INFO: Query finished. [astroquery.utils.tap.core]
what 8
INFO: Query finished. [astroquery.utils.tap.core]
9 2 255.30638288363912 -30.108883584929643
INFO: Query finished. [astroquery.utils.tap.core]
10 2 265.2041279091693 -53.66356128324382
INFO: Query finished. [astroquery.utils.tap.core]
11 1 265.2888155609892 -53.58671614187193
INFO

In [44]:
df.to_csv(path+'match_GC_GAIA_3arcsec.csv', index=False)

In [46]:
Gaia.ROW_LIMIT=-1
coord = SkyCoord(ra=265.17538, dec=-53.67433, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(20, u.arcmin)
height = u.Quantity(20, u.arcmin)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)
df = pd.DataFrame(np.array(r), columns=r.columns)
path='/Users/baotong/Desktop/period_terzan5/CSC/'
df.to_csv(path+'GAIA_NGC6397.csv', index=False)

INFO: Query finished. [astroquery.utils.tap.core]
